In [1]:
import numpy as np 
import pandas as pd 
import os
from glob import glob
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
from skimage import io

from keras.preprocessing.image import ImageDataGenerator
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.models import Sequential, Model
from keras.applications.vgg16 import VGG16
from keras.applications.resnet import ResNet50 
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau

Using TensorFlow backend.


In [2]:
train_df = pd.read_csv('train.csv')
valid_df = pd.read_csv('test.csv')

In [3]:
train_df.head()

,img_path,class
0,mdb003.pgm.png,dense
1,mdb004.pgm.png,dense
2,mdb033.pgm.png,dense
3,mdb034.pgm.png,dense
4,mdb035.pgm.png,dense


In [4]:
train_df["class"].value_counts()

dense    10
fatty    10
Name: class, dtype: int64

In [5]:
valid_df["class"].value_counts()

dense    3
fatty    3
Name: class, dtype: int64

## Setting up the image augmentation from last Lesson: 
Note that this section of the code has been pre-written for you and does not need to be changed, just run. If you would like to change the ImageDataGenerator parameters, feel free.

In [6]:
## This is the image size that VGG16 takes as input
IMG_SIZE = (224, 224)

In [7]:
train_idg = ImageDataGenerator(rescale=1. / 255.0,
                              horizontal_flip = True, 
                              vertical_flip = False, 
                              height_shift_range= 0.1, 
                              width_shift_range=0.1, 
                              rotation_range=20, 
                              shear_range = 0.1,
                              zoom_range=0.1)

train_gen = train_idg.flow_from_dataframe(dataframe=train_df, 
                                         directory=None, 
                                         x_col = 'img_path',
                                         y_col = 'class',
                                         class_mode = 'binary',
                                         target_size = IMG_SIZE, 
                                         batch_size = 9
                                         )

# Note that the validation data should not be augmented! We only want to do some basic intensity rescaling here
val_idg = ImageDataGenerator(rescale=1. / 255.0
                                 )

val_gen = val_idg.flow_from_dataframe(dataframe=valid_df, 
                                         directory=None, 
                                         x_col = 'img_path',
                                         y_col = 'class',
                                         class_mode = 'binary',
                                         target_size = IMG_SIZE, 
                                         batch_size = 6) ## We've only been provided with 6 validation images

Found 20 validated image filenames belonging to 2 classes.
Found 6 validated image filenames belonging to 2 classes.


In [8]:
## Pull a single large batch of random validation data for testing after each epoch
testX, testY = val_gen.next()

## Now we'll load in VGG16 with pre-trained ImageNet weights: 

In [9]:
model = VGG16(include_top=True, weights='imagenet')
model.summary()

553467904/553467096 [==============================] - 7s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
____________________________________________________________

In [10]:
transfer_layer = model.get_layer('block5_pool')
vgg_model = Model(inputs=model.input,
                   outputs=transfer_layer.output)

In [11]:
for i, layer in enumerate(vgg_model.layers):
    print(i, layer)

0 <keras.engine.input_layer.InputLayer object at 0x7f529f8bbd10>
1 <keras.layers.convolutional.Conv2D object at 0x7f529f800490>
2 <keras.layers.convolutional.Conv2D object at 0x7f529f800210>
3 <keras.layers.pooling.MaxPooling2D object at 0x7f529f8ccdd0>
4 <keras.layers.convolutional.Conv2D object at 0x7f529f8cc750>
5 <keras.layers.convolutional.Conv2D object at 0x7f529f88ef90>
6 <keras.layers.pooling.MaxPooling2D object at 0x7f529f89af50>
7 <keras.layers.convolutional.Conv2D object at 0x7f529f89a850>
8 <keras.layers.convolutional.Conv2D object at 0x7f529f89eb50>
9 <keras.layers.convolutional.Conv2D object at 0x7f529f8a3f90>
10 <keras.layers.pooling.MaxPooling2D object at 0x7f529f8a3f50>
11 <keras.layers.convolutional.Conv2D object at 0x7f529f7a7a90>
12 <keras.layers.convolutional.Conv2D object at 0x7f529f7ae850>
13 <keras.layers.convolutional.Conv2D object at 0x7f529f7b3d10>
14 <keras.layers.pooling.MaxPooling2D object at 0x7f529f7b8d10>
15 <keras.layers.convolutional.Conv2D object at 

In [13]:
## Now, choose which layers of VGG16 we actually want to fine-tune
## Here, we'll freeze all but the last convolutional layer

## Add some code here to freeze all but the last convolutional layer:
##### Your code here ######

for layer in vgg_model.layers[:17]:
    layer.trainable = False

In [14]:
## Check to make sure you froze the right ones: 
for layer in vgg_model.layers:
    print(layer.name, layer.trainable)

input_1 False
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool False
block4_conv1 False
block4_conv2 False
block4_conv3 False
block4_pool False
block5_conv1 False
block5_conv2 False
block5_conv3 True
block5_pool True


## Build a simple sequential model using only the VGG16 architecture
Note the code in the cell below has been pre-written for you, you only need to run it

In [15]:
## Build your model using the mostly-frozen VGG16 architecture: 
new_model = Sequential()

# Add the convolutional part of the VGG16 model from above.
new_model.add(vgg_model)

# Flatten the output of the VGG16 model because it is from a
# convolutional layer.
new_model.add(Flatten())

# Add a dense (aka. fully-connected) layer.
# This is for combining features that the VGG16 model has
# recognized in the image.
new_model.add(Dense(1, activation='relu'))

In [16]:
## Set our optimizer, loss function, and learning rate (you can change the learning rate here if you'd like)
## but otherwise this cell can be run as is
optimizer = Adam(lr=1e-4)
loss = 'binary_crossentropy'
metrics = ['binary_accuracy']

In [17]:
new_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [18]:
## Just run a single epoch to see how it does:
new_model.fit_generator(train_gen,
                        validation_data = (testX, testY), 
                        epochs = 5)

Epoch 1/5
3/3 [==============================] - 17s 6s/step - loss: 1.4751 - binary_accuracy: 0.4500 - val_loss: 1.1110 - val_binary_accuracy: 0.5000
Epoch 2/5
3/3 [==============================] - 16s 5s/step - loss: 0.9680 - binary_accuracy: 0.5500 - val_loss: 0.7657 - val_binary_accuracy: 0.5000
Epoch 3/5
3/3 [==============================] - 15s 5s/step - loss: 0.9385 - binary_accuracy: 0.6000 - val_loss: 0.8860 - val_binary_accuracy: 0.6667
Epoch 4/5
3/3 [==============================] - 15s 5s/step - loss: 0.4250 - binary_accuracy: 0.6500 - val_loss: 0.6256 - val_binary_accuracy: 0.6667
Epoch 5/5
3/3 [==============================] - 15s 5s/step - loss: 0.3950 - binary_accuracy: 0.7500 - val_loss: 0.5849 - val_binary_accuracy: 0.6667


## Let's try another experiment where we add a few more dense layers:

In [19]:
new_model = Sequential()

# Add the convolutional part of the VGG16 model from above.
new_model.add(vgg_model)

# Flatten the output of the VGG16 model because it is from a
# convolutional layer.
new_model.add(Flatten())

# Add a couple of dense (aka. fully-connected) layers.
# This is for combining features that the VGG16 model has
# recognized in the image.

##### Your code here ######
new_model.add(Dense(32, activation='relu'))

# Final output layer:
new_model.add(Dense(1, activation='relu'))

In [20]:
new_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [21]:
## Just run a single epoch to see how it does:
new_model.fit_generator(train_gen,
                        validation_data = (testX, testY), 
                        epochs = 5)

Epoch 1/5
3/3 [==============================] - 16s 5s/step - loss: 6.2128 - binary_accuracy: 0.5000 - val_loss: 7.7125 - val_binary_accuracy: 0.5000
Epoch 2/5
3/3 [==============================] - 16s 5s/step - loss: 7.7125 - binary_accuracy: 0.5000 - val_loss: 7.7125 - val_binary_accuracy: 0.5000
Epoch 3/5
3/3 [==============================] - 16s 5s/step - loss: 7.7125 - binary_accuracy: 0.5000 - val_loss: 7.7125 - val_binary_accuracy: 0.5000
Epoch 4/5
3/3 [==============================] - 16s 5s/step - loss: 6.2128 - binary_accuracy: 0.5000 - val_loss: 7.7125 - val_binary_accuracy: 0.5000
Epoch 5/5
3/3 [==============================] - 16s 5s/step - loss: 7.7125 - binary_accuracy: 0.5000 - val_loss: 7.7125 - val_binary_accuracy: 0.5000


## Now let's add dropout and another fully connected layer:

In [22]:
new_model = Sequential()

# Add the convolutional part of the VGG16 model from above.
new_model.add(vgg_model)

# Flatten the output of the VGG16 model because it is from a
# convolutional layer.
new_model.add(Flatten())

# Add several fully-connected layers with dropout

##### Your code here ######
new_model.add(Dropout(0.25))

new_model.add(Dense(32, activation='relu'))

# Final output layer:
new_model.add(Dense(1, activation='relu'))

In [23]:
new_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [24]:
## Just run a single epoch to see how it does:
new_model.fit_generator(train_gen,
                        validation_data = (testX, testY), 
                        epochs = 5)

Epoch 1/5
3/3 [==============================] - 16s 5s/step - loss: 7.1415 - binary_accuracy: 0.5000 - val_loss: 7.7125 - val_binary_accuracy: 0.5000
Epoch 2/5
3/3 [==============================] - 16s 5s/step - loss: 5.1976 - binary_accuracy: 0.5000 - val_loss: 7.7125 - val_binary_accuracy: 0.5000
Epoch 3/5
3/3 [==============================] - 16s 5s/step - loss: 7.7142 - binary_accuracy: 0.5000 - val_loss: 7.7125 - val_binary_accuracy: 0.5000
Epoch 4/5
3/3 [==============================] - 15s 5s/step - loss: 7.7125 - binary_accuracy: 0.5000 - val_loss: 7.7125 - val_binary_accuracy: 0.5000
Epoch 5/5
3/3 [==============================] - 15s 5s/step - loss: 9.2121 - binary_accuracy: 0.5000 - val_loss: 7.7125 - val_binary_accuracy: 0.5000
